# Description
Let's see what properties we can find :) <br>
Let's put more data into it !

## TODOs
 - scale frequency spectrum before
 - species level precision (instead of strand level)
 - automate all : 
  - PCA/LDA/Autoencoder/nothing
  - number of bacteria and samples
 - treat plasmids differently
 - try with real Nanopore reads
 - unsupervised learning on all windows into 10 bins ?
 - hierarchical clustering ??
 - clustering with minimal distance
 - taxo id to family or genus
 - tsne plot
 - Add **Visualisation** : https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html
 - Save each case, not each run. don't run cases that have already been computed.

## Structure
In `genome/`, there's multiple sub-folder, we will start with `Bacteria`
It then contains all recorded species/strands in individual folders


http://defindit.com/readme_files/ncbi_file_extension_format.html

What we need is the taxo id, name, and the DNA, which can be found in:
 - .gbk for the taxo and name
 - .fna for the sequence

# Import and Paths

In [1]:
print("let's start")

let's start


In [90]:
import os
import pandas as pd
import numpy as np
import pickle
import traceback
import multiprocessing
import random
import pickle
from collections import Counter
from datetime import datetime as dt
from time import time
from tqdm import tqdm_notebook as tqdm

In [3]:
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

In [4]:
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
n_jobs = multiprocessing.cpu_count()

In [6]:
plt.rcParams['figure.figsize'] = 13, 8

In [7]:
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
path_ref_db = "/home/ubuntu/Data/NCBI/Bacteria_2015/"
path_kmer_freq = "/home/ubuntu/Data/kmer_freq/"

In [9]:
path_4mer = "4mer/V4/"
path_4mer = os.path.join(path_kmer_freq, path_4mer)

In [10]:
path_all = os.path.join(path_4mer, "_all_bacteria_4mers.largepd")
print(f"{os.path.isfile(path_all)} path_all")

True path_all


# Functions

In [11]:
nucleotides = "ACGT"

In [12]:
def read_fna(file_path):
    with open(file_path) as f:
        rec = f.readlines()
        return "".join(rec[1:]).replace("\n", "")

In [13]:
def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [f"{a}{n}" for a in combinaisons(combi, n-1) for n in instances]

In [14]:
def kmers_dic(n, choice=nucleotides):
    return {a:0 for a in combinaisons(choice, n)}

In [15]:
def window(fseq, window_size=4):
    for i in range(len(fseq) - window_size + 1):
        yield fseq[i:i+window_size]

In [16]:
def count_all_kmers(seq, kmer_template, n, ignore_N=True):
    """ Count all kmers, ignore kmers with N or other undecided nucleotides 
        Return a list of dict for each window (w=100)
        seq: string nucleotide input
        kmer_template: dict with all combinations of nucleotides, initialized with zeros (kmer_template["AAAA"] = 0, kmer_template["AAAC"] = 0...)
    """
    kmer_counts = kmer_template.copy()
    wrong_base = "N"*n
    kmer_counts[wrong_base] = 0
    
    try:
        for i, kmer in enumerate(window(seq, n)):
            try:
                kmer_counts[kmer] += 1
            except:
                kmer_counts[wrong_base] += 1
                
        if ignore_N:
            kmer_counts.pop(wrong_base)
        return kmer_counts
    except Exception as e:
        print("type error: " + str(e))
        print(traceback.format_exc())
        return kmer_counts

In [17]:
def count_kmers(seq, kmer_template, n, bacteria_name, fna, w=100):
    """ Count all kmers, ignore kmers with N or other undecided nucleotides 
        Return a list of dict for each window (w=100)
    """
    res = []
    current_split = 0
    next_split = current_split + w
    tmp_counts = kmer_template.copy()
    tmp_counts["start"] = current_split
    
    try:
        for i, kmer in enumerate(window(seq, n)):
            try:
                tmp_counts[kmer] += 1
            except:
                pass
            # To lower the computational need to split into windows
            if i == next_split:
                res.append(tmp_counts)
                current_split = next_split
                next_split += w
                tmp_counts = kmer_template.copy()
                tmp_counts["start"] = current_split
                
        return i+1, res
    except Exception as e:
        print("type error: " + str(e))
        print(traceback.format_exc())
        return i, res

In [18]:
def kmer_pkl_path(kmer_folder, fna_path):
    """ Return a file name based on the taxonomy id instead of the file name.
        The taxo id is looked for in the associated .gbk file.
    """
    path_gbk = fna_path.replace(".fna", ".gbk")
    assert os.path.isfile(path_gbk), f"{fna_path} DOESN'T have a .gbk file ??"
    
    with open(path_gbk) as gbk:
        description=gbk.read()  #.replace('\n', '')
        
    identificator = 'db_xref="taxon:'
    taxo_start = description.find(identificator)
    taxo = description[taxo_start+len(identificator):
                       taxo_start+description[taxo_start:].find('"\n')]
    assert len(taxo) < 10, f"The taxo id search failed, found an id of length {len(taxo)}..."
    
    bacteria_name = os.path.split(os.path.split(fna_path)[0])[1]
    fna_name      = os.path.split(os.path.splitext(fna_path)[0])[1]
    
    out_path = os.path.join(path_kmer_freq, kmer_folder, f"{taxo}__{bacteria_name}.pd")
    
    return taxo, bacteria_name, fna_name, out_path

# Create reads

## Synthetic reads from the Ref Database
find reads, real or from ncbi
only genome, not plasmid ?

In [19]:
nb_reads = 10000
seed = 34
nb_species = len(os.listdir(path_ref_db))

In [36]:
def reads_from_ncbi_genomes():
    fake_reads = {"bac": [], "seq": []}
    nb_species = len(os.listdir(path_ref_db))
    indices = sorted(np.random.choice(range(nb_species), nb_reads))
    indices_multi = Counter(indices)
    
    for i, file_name in tqdm(enumerate(os.listdir(path_ref_db)), total=nb_species):
        if i in indices_multi.keys():
    #         print(i, indices_multi[i], file_name)
            try:
                fake_reads["bac"].extend([file_name]*indices_multi[i])
                folder_bacteria = os.path.join(path_ref_db, file_name)
                all_fna = sorted([file.path for file in os.scandir(folder_bacteria) if file.name.endswith(".fna")])
                all_fna_sizes = [os.path.getsize(f) for f in all_fna]
                max_value = max(all_fna_sizes)
                max_index = all_fna_sizes.index(max_value)
                rec = read_fna(all_fna[max_index]) 

                rec_len = len(rec)
                reads_len = [random.randint(1000, min(20000, rec_len)) for k in range(indices_multi[i])]
                reads_start = [random.randint(0, rec_len-reads_len[k]) for k in range(indices_multi[i])]

                for j in range(indices_multi[i]):
                    fake_reads["seq"].append(rec[reads_start[j] : reads_start[j]+reads_len[j]])
    #                 print(reads_start[j], reads_start[j]+reads_len[j], rec_len, len(fake_reads["seq"][-1]))
            except Exception as e:
                print(folder_bacteria)
                print(e)
                
    return fake_reads

In [37]:
fake_reads = reads_from_ncbi_genomes()

In [38]:
save_fake_reads = fake_reads.copy()

In [61]:
fake_reads = save_fake_reads.copy()

## Real reads from isolates
todo

## Save data

# kmer count
All random sequences from NCBI have been generated <br>
Now, count their kmer frequencies

In [88]:
k = 4
kmer4 = kmers_dic(k)
cols_kmers = list(kmer4.keys())

In [65]:
for key in kmer4.keys():
    fake_reads[key] = []

Count and add counts besides the Dataframe

In [68]:
for i in tqdm(range(len(fake_reads["bac"]))):
    counts = count_all_kmers(fake_reads["seq"][i], kmer4, k)
    for key in kmer4.keys():
        fake_reads[key].append(counts[key])

In [89]:
len(fake_reads["seq"]), len(fake_reads["bac"]), len(fake_reads["AAAA"])

(10000, 10000, 10000)

In [70]:
pd_reads = pd.DataFrame(fake_reads)

In [71]:
pd_reads.head(5)

,bac,seq,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,Acaryochloris_marina_MBIC11017_uid58167,AACGATGAAAATTTTTAGCTTTATTTAGAGGTTGATATCAAAAATA...,99,101,81,104,53,98,64,69,...,55,77,121,125,95,63,66,92,130,113
1,Acaryochloris_marina_MBIC11017_uid58167,TGGCAAAATCGTCGGAATTTCTGGTTCCCTGGCGGACGTAACCGAA...,211,104,119,189,114,86,44,93,...,63,117,129,95,121,89,136,124,134,169
2,Acaryochloris_marina_MBIC11017_uid58167,GCAGGGGAAACTGCCGTAAATAATTGTCTTTGGTCGTTAAAGGTAG...,165,78,131,161,81,76,63,87,...,69,126,169,117,133,87,109,89,143,144
3,Acetobacterium_woodii_DSM_1030_uid88073,ATGTGTCGTTGCTTGTCCGGTTGGGAAGAAGGCAACCGCCCCCAAA...,74,28,32,49,15,21,15,17,...,7,13,29,15,18,25,43,15,20,49
4,Acetobacterium_woodii_DSM_1030_uid88073,TAATGCCTCAATTGTTTTAAGTAAGTTCTGGTAGGAAACCTGAAAA...,58,41,37,66,30,51,27,18,...,39,55,43,32,29,29,63,103,50,135


In [72]:
pd_reads.shape

(10000, 258)

# Reload and apply the LDA model

LDA trained on reference genomes

In [73]:
file_lda_model = f"/home/ubuntu/Data/kmer_freq/4mer/V4/LDA/lda_model_20_int.pd"
lda = pickle.load(open(file_lda_model, 'rb'))

In [74]:
components = 20
np_lda = lda.transform(pd_reads[cols_kmers])
cols_lda = [f"lda_{i+1}" for i in range(components)]
df_lda = pd.DataFrame(np_lda, columns=cols_lda)
df_lda = pd.concat([pd_reads["bac"].reset_index(inplace=False, drop=True), df_lda], axis=1) # , ignore_index=True)

In [75]:
df_lda.head(5)

,bac,lda_1,lda_2,lda_3,lda_4,lda_5,lda_6,lda_7,lda_8,lda_9,...,lda_11,lda_12,lda_13,lda_14,lda_15,lda_16,lda_17,lda_18,lda_19,lda_20
0,Acaryochloris_marina_MBIC11017_uid58167,2.16,-0.28,5.55,-10.76,4.89,-6.12,2.53,7.11,-0.47,...,-1.51,1.14,-6.35,-4.26,2.56,1.34,-0.16,4.96,-9.48,-1.58
1,Acaryochloris_marina_MBIC11017_uid58167,7.69,1.91,9.66,-8.39,6.02,-3.14,-0.82,5.56,-5.09,...,-1.16,-2.88,-3.52,-6.08,2.89,-2.78,4.44,7.40,-8.57,-0.81
2,Acaryochloris_marina_MBIC11017_uid58167,5.18,1.26,11.39,-10.15,5.89,-4.16,-0.13,6.67,-2.31,...,-0.70,-2.40,-3.01,-5.69,4.04,-1.07,1.68,6.49,-7.73,-3.51
3,Acetobacterium_woodii_DSM_1030_uid88073,1.29,-0.41,-3.68,2.16,-1.18,-2.08,-1.61,-0.32,0.41,...,0.51,2.72,1.40,3.36,-2.25,1.26,0.82,-4.36,4.00,-0.97
4,Acetobacterium_woodii_DSM_1030_uid88073,4.01,-1.99,-1.75,-0.49,-1.00,-3.18,-2.52,-0.71,0.82,...,0.93,1.75,-0.82,1.49,-0.19,-1.04,0.92,-0.27,-1.69,-0.39


## Scaling

In [76]:
df_lda[cols_lda] = scale(df_lda[cols_lda])

# Load classifier and bin reads

In [77]:
file_NB_model = f"/home/ubuntu/Data/kmer_freq/4mer/V4/ml_models/_2782_Naive_Bayes_2019-05-08_17-24.pkl"
naive_bayes = pickle.load(open(file_NB_model, 'rb'))

In [78]:
pred = naive_bayes.predict(df_lda[cols_lda])

In [79]:
df_lda["pred"] = pred

In [80]:
df_lda["genus_match"] = (df_lda.bac.str.split("_", 1).str[0] ==
                         df_lda[f"pred"].str.split("_", 1).str[0]).astype(int)

In [86]:
matches = df_lda[["genus_match"]].sum()
print(f"{matches[0]} matches, {100*matches[0]/df_lda.shape[0]:.1f}%")

4951 matches, 49.5%


# Binning by clustering

In [91]:
with open(f"/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/10_agglo_2019-05-09_04-08.pkl", 'rb') as f: 
    agglo = pickle.load(f)

In [92]:
with open(f"/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/10_kmeans_2019-05-09_04-08.pkl", 'rb') as f: 
    kmeans = pickle.load(f)

# End of the script.
Sylvain @GIS